In [1]:
import os
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, LSTM
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.models import model_from_json
from keras.callbacks import EarlyStopping, History
from keras.optimizers import SGD
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import random
import logging
from functools import reduce
from operator import add
import matplotlib.pyplot as plt

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.75
#config.allow_soft_placement = True
#config.log_device_placement = True
set_session(tf.Session(config = config))

%matplotlib inline

C:\Users\vales\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
train = pd.read_csv('D:/data/numerai/numerai_training_data.csv')
tournament = pd.read_csv('D:/data/numerai/numerai_tournament_data.csv')

In [3]:
#pd.get_dummies(train['era'])
combo = pd.concat((train, tournament), axis = 0)
tournament_len = len(tournament)

era_dummies = pd.get_dummies(combo['era'])

combo_dummies = pd.concat((combo, era_dummies), axis = 1)

train = combo_dummies.iloc[:-(tournament_len)]
tournament = combo_dummies.iloc[-(tournament_len):]

In [54]:
#train.to_csv('D:/data/numerai/train_proccessed_5_16.csv', index = False)
#tournament.to_csv('D:/data/numerai/tournament_processed_5_16.csv', index = False)
for col in train.columns:
    if col == 'target':
        print('si')

si


In [5]:
def return_train_cv_set(train, cv_len):    
    num_eras = len(train['era'].unique())
    
    cv_eras_ls = []

    for _ in range(cv_len):
        x = random.randint(1,num_eras)

        cv_eras_ls.append('era' + str(x))
    
    train_indices = list(train.index)
    
    cv_indices = []
    
    for era in cv_eras_ls:
        temp_indices = train.index[(train['era'] == era)]
        cv_indices.extend(temp_indices)
    
    cv_set = train.ix[cv_indices]

    for cv_index in cv_indices:
        train_indices.remove(cv_index)

    train_no_cv = train.ix[train_indices]
        
    return cv_set, train_no_cv

def return_X_y(df):
    seed = 19

    cols = list(df.columns)
    cols.remove('target')
    cols = cols[3:]

    X = df[cols]
    y = df['target']
    
    X = shuffle(X, random_state = seed)
    y = shuffle(y, random_state = seed)
    
    return X.as_matrix(), y.as_matrix()

In [6]:
#cv_set, train_no_cv = return_train_cv_set(train, 10)
train_X, train_y = return_X_y(train)
#cv_X, cv_y = return_X_y(cv_set)
input_shape = ((train_X.shape[1],))

In [16]:
def compile_model(network, input_shape):
    # Create and compile Model

    # Args: network (dict) of params

    nb_layers = network['nb_layers']
    nb_neurons = network['nb_neurons']
    activation = network['activation']
    optimizer = network['optimizer']
    dropout_percent = network['dropout_percent']

    model = Sequential()

    for l in range(nb_layers):
        if l % 3 == 0:
            model.add(Dense(nb_neurons, activation = 'relu', kernel_initializer = 'normal', input_shape = input_shape))
            model.add(Dropout(dropout_percent))
        else:
            model.add(Dense(nb_neurons, activation = activation, kernel_initializer = 'normal'))

    model.add(Dense(1, kernel_initializer = 'normal', activation = 'sigmoid'))
    
    sgd = SGD(lr= .005)
    model.compile(loss = 'binary_crossentropy', optimizer = sgd)

    return model


def train_and_score_single(network, train_X, train_y):# cv_X, cv_y):
    #early_stopper = EarlyStopping(patience = 20)
    history = History()
    
    
    #model = KerasRegressor(build_fn = compile_model, epochs = 2500, batch_size = 2400, verbose = 1, 
    #                       validation_split = network['validation_split'], callbacks = [history],
    #                      network = network, input_shape = input_shape)
    
    model = compile_model(network, input_shape)
    
    model.fit(train_X, train_y, epochs = 3500, batch_size = 3000, verbose = 1, 
              callbacks = [history])
    
    #score = model.evaluate(cv_X_scaled, cv_y, verbose = 0)
    
    return model, history


In [ ]:
final_network = {'nb_layers': 15, 'validation_split': 0.25, 'nb_neurons': 512, 'optimizer': 'adam', 'activation': 'sigmoid',
                'dropout_percent': 0.20}
model, history = train_and_score_single(final_network, train_X, train_y)# cv_X, cv_y)

Epoch 1/3500
108405/108405 [==============================] - ETA: 27s - loss: 0.73 - ETA: 14s - loss: 0.73 - ETA: 10s - loss: 0.72 - ETA: 8s - loss: 0.7184 - ETA: 7s - loss: 0.714 - ETA: 6s - loss: 0.712 - ETA: 5s - loss: 0.710 - ETA: 5s - loss: 0.708 - ETA: 4s - loss: 0.706 - ETA: 4s - loss: 0.705 - ETA: 3s - loss: 0.704 - ETA: 3s - loss: 0.703 - ETA: 3s - loss: 0.702 - ETA: 3s - loss: 0.702 - ETA: 2s - loss: 0.701 - ETA: 2s - loss: 0.700 - ETA: 2s - loss: 0.700 - ETA: 2s - loss: 0.700 - ETA: 2s - loss: 0.699 - ETA: 2s - loss: 0.699 - ETA: 1s - loss: 0.699 - ETA: 1s - loss: 0.698 - ETA: 1s - loss: 0.698 - ETA: 1s - loss: 0.698 - ETA: 1s - loss: 0.698 - ETA: 1s - loss: 0.697 - ETA: 1s - loss: 0.697 - ETA: 0s - loss: 0.697 - ETA: 0s - loss: 0.697 - ETA: 0s - loss: 0.697 - ETA: 0s - loss: 0.697 - ETA: 0s - loss: 0.697 - ETA: 0s - loss: 0.696 - ETA: 0s - loss: 0.696 - ETA: 0s - loss: 0.696 - ETA: 0s - loss: 0.696 - 4s - loss: 0.6965     
Epoch 2/3500
108405/108405 [======================

108405/108405 [==============================] - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - 3s - loss: 0.6932     
Epoch 18/3500
108405/108405 [==============================] - E

108405/108405 [==============================] - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - 3s - loss: 0.6932     
Epoch 34/3500
108405/108405 [==============================] - E

108405/108405 [==============================] - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 3s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 2s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 1s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - ETA: 0s - loss: 0.693 - 3s - loss: 0.6932     
Epoch 50/3500
108405/108405 [==============================] - E

In [ ]:
# create model

history = History()
simple_model = Sequential()
simple_model.add(Dense(1024, input_shape = input_shape, activation='relu'))
simple_model.add(Dense(512, activation='relu'))
simple_model.add(Dense(1, activation='sigmoid'))

simple_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
simple_model.fit(train_X, train_y, epochs = 700, batch_size = 2400, verbose = 1, validation_data = (cv_X, cv_y), 
              callbacks = [history])

In [105]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.ylim(0,0.004)
plt.show()

((97588,), (10817,))

In [52]:
for i in range(len(cv_y)):
    print(cv_y[i], cv_X[i,:5])

1.0 [ 0.56833  0.60516  0.50622  0.56455  0.50227]
1.0 [ 0.5088   0.56095  0.57715  0.52208  0.46215]
0.0 [ 0.50141  0.55132  0.59212  0.48361  0.4111 ]
0.0 [ 0.54243  0.58943  0.52222  0.54705  0.49329]
0.0 [ 0.50942  0.56422  0.53584  0.48637  0.53012]
0.0 [ 0.52112  0.45552  0.74799  0.30045  0.21477]
0.0 [ 0.41599  0.47814  0.70806  0.33771  0.31601]
1.0 [ 0.49536  0.60526  0.6224   0.50286  0.38674]
1.0 [ 0.49874  0.5646   0.54046  0.49607  0.50859]
1.0 [ 0.56711  0.38231  0.68981  0.27056  0.26698]
0.0 [ 0.46836  0.55038  0.49732  0.5642   0.59611]
1.0 [ 0.45587  0.46649  0.67479  0.37145  0.34827]
0.0 [ 0.61591  0.4485   0.60252  0.38379  0.34413]
0.0 [ 0.49466  0.55286  0.59858  0.46255  0.42315]
0.0 [ 0.38254  0.50072  0.73888  0.4223   0.28843]
1.0 [ 0.53433  0.5582   0.56206  0.49583  0.42908]
1.0 [ 0.55716  0.46922  0.65125  0.35845  0.31396]
0.0 [ 0.52356  0.4849   0.6073   0.38709  0.41111]
1.0 [ 0.38129  0.58055  0.63963  0.48727  0.44265]
1.0 [ 0.46288  0.5409   0.66281

1.0 [ 0.50825  0.66035  0.5286   0.50614  0.47632]
1.0 [ 0.50862  0.57249  0.52457  0.49695  0.52279]
0.0 [ 0.57476  0.6468   0.52903  0.59728  0.46116]
1.0 [ 0.52849  0.54351  0.57981  0.51194  0.42746]
1.0 [ 0.52568  0.41783  0.70193  0.3283   0.28179]
1.0 [ 0.47981  0.634    0.5454   0.55554  0.52139]
1.0 [ 0.50257  0.48     0.71996  0.30408  0.24946]
0.0 [ 0.55868  0.52318  0.57842  0.5129   0.409  ]
0.0 [ 0.44972  0.63205  0.60088  0.47927  0.43147]
1.0 [ 0.58464  0.59785  0.49022  0.60281  0.5212 ]
0.0 [ 0.60033  0.58851  0.50894  0.51135  0.46917]
0.0 [ 0.75851  0.30862  0.56494  0.43877  0.34388]
0.0 [ 0.38067  0.53525  0.67371  0.44049  0.39108]
0.0 [ 0.56675  0.3499   0.60954  0.3573   0.38398]
0.0 [ 0.51249  0.63595  0.56833  0.49016  0.43597]
1.0 [ 0.58438  0.59892  0.42838  0.62678  0.6102 ]
0.0 [ 0.52783  0.72328  0.53472  0.55407  0.46166]
1.0 [ 0.38077  0.49217  0.744    0.39884  0.28866]
1.0 [ 0.30413  0.40091  0.72108  0.38821  0.40689]
0.0 [ 0.63607  0.61881  0.42084

1.0 [ 0.51048  0.51949  0.58722  0.50421  0.43449]
1.0 [ 0.46435  0.61451  0.49238  0.58309  0.5682 ]
1.0 [ 0.48582  0.52115  0.62113  0.46352  0.37431]
0.0 [ 0.48469  0.51528  0.6202   0.4315   0.3922 ]
0.0 [ 0.5496   0.48462  0.61722  0.43579  0.3725 ]
1.0 [ 0.39708  0.38834  0.76772  0.25047  0.26366]
0.0 [ 0.49226  0.53138  0.58768  0.43797  0.45395]
1.0 [ 0.46118  0.68645  0.49152  0.61759  0.56938]
1.0 [ 0.47645  0.54112  0.61848  0.47769  0.42875]
1.0 [ 0.48689  0.70042  0.49304  0.51921  0.55255]
1.0 [ 0.43618  0.47917  0.68176  0.42419  0.34477]
1.0 [ 0.5281   0.49078  0.56656  0.40492  0.44879]
0.0 [ 0.59918  0.4789   0.52617  0.44061  0.47064]
0.0 [ 0.53836  0.60688  0.54972  0.52669  0.45953]
0.0 [ 0.50196  0.52886  0.60372  0.43861  0.41444]
0.0 [ 0.14989  0.2804   0.97399  0.16074  0.14905]
0.0 [ 0.54012  0.54043  0.57273  0.46578  0.43231]
0.0 [ 0.50621  0.51396  0.61734  0.47726  0.38289]
1.0 [ 0.51073  0.51638  0.5928   0.43543  0.41637]
0.0 [ 0.47315  0.58288  0.5792 

0.0 [ 0.27121  0.52537  0.74973  0.38295  0.34735]
0.0 [ 0.53795  0.57094  0.60595  0.43567  0.3719 ]
0.0 [ 0.49371  0.61081  0.60778  0.46664  0.38631]
0.0 [ 0.60059  0.69709  0.37913  0.60635  0.64784]
1.0 [ 0.46917  0.43089  0.71385  0.38888  0.28365]
1.0 [ 0.53871  0.65929  0.48916  0.56566  0.51763]
0.0 [ 0.40787  0.46092  0.65317  0.3263   0.40124]
0.0 [ 0.48527  0.59653  0.58057  0.49243  0.43877]
1.0 [ 0.46632  0.56814  0.56186  0.49833  0.48621]
1.0 [ 0.55071  0.49938  0.62679  0.41578  0.35582]
0.0 [ 0.28665  0.49175  0.73571  0.3784   0.37691]
0.0 [ 0.36541  0.46502  0.71605  0.36599  0.36061]
1.0 [ 0.52915  0.60558  0.52203  0.56004  0.49398]
1.0 [ 0.49427  0.52151  0.62871  0.42391  0.37295]
1.0 [ 0.51546  0.50787  0.6174   0.44652  0.36832]
1.0 [ 0.50129  0.50043  0.59914  0.48375  0.42573]
0.0 [ 0.45616  0.61712  0.60459  0.48759  0.42034]
1.0 [ 0.52904  0.48366  0.63279  0.47566  0.35125]
0.0 [ 0.43943  0.43068  0.63987  0.44549  0.43112]
0.0 [ 0.5926   0.58797  0.5091 

1.0 [ 0.49089  0.49939  0.53812  0.52339  0.53629]
0.0 [ 0.36208  0.59285  0.61011  0.4623   0.48359]
0.0 [ 0.31943  0.52627  0.76068  0.36711  0.31148]
0.0 [ 0.46328  0.44578  0.58911  0.45437  0.46759]
0.0 [ 0.28761  0.43953  0.74872  0.39931  0.36801]
0.0 [ 0.55701  0.68221  0.47528  0.5681   0.52654]
1.0 [ 0.52664  0.5715   0.54443  0.53186  0.47772]
1.0 [ 0.4448   0.35001  0.61509  0.37475  0.47748]
1.0 [ 0.38129  0.48954  0.69593  0.39468  0.37466]
0.0 [ 0.45663  0.59818  0.54101  0.46475  0.52644]
1.0 [ 0.43146  0.42286  0.67776  0.32335  0.36915]
0.0 [ 0.61218  0.56766  0.55645  0.46683  0.39967]
0.0 [ 0.52666  0.38655  0.65818  0.38425  0.36604]
1.0 [ 0.50747  0.49485  0.60374  0.43974  0.42466]
1.0 [ 0.25949  0.31127  0.88121  0.18507  0.18605]
1.0 [ 0.53639  0.54896  0.56999  0.51544  0.43862]
0.0 [ 0.54955  0.72368  0.50248  0.55971  0.49574]
1.0 [ 0.63127  0.592    0.46979  0.56687  0.51976]
0.0 [ 0.37813  0.41989  0.67934  0.41569  0.43115]
0.0 [ 0.47861  0.61017  0.60897

...

0.0 [ 0.48373  0.53231  0.63676  0.45164  0.38883]
1.0 [ 0.47387  0.55876  0.59137  0.49127  0.44149]
0.0 [ 0.48432  0.5235   0.61506  0.44881  0.41386]
0.0 [ 0.462    0.34453  0.68528  0.31656  0.35523]
0.0 [ 0.51715  0.48268  0.54585  0.51894  0.4997 ]
0.0 [ 0.55903  0.53483  0.50451  0.58218  0.51983]
1.0 [ 0.55384  0.39241  0.62264  0.39652  0.37803]
0.0 [ 0.25906  0.29449  0.88468  0.23047  0.20266]
0.0 [ 0.52111  0.56258  0.60251  0.43691  0.40936]
1.0 [ 0.62766  0.54159  0.54305  0.44369  0.42783]
0.0 [ 0.41811  0.55652  0.5829   0.47713  0.49993]
1.0 [ 0.45154  0.54303  0.61426  0.42702  0.40936]
0.0 [ 0.43651  0.5497   0.63812  0.47278  0.4133 ]
0.0 [ 0.70092  0.6498   0.36781  0.61388  0.61346]
0.0 [ 0.56124  0.52614  0.55199  0.50882  0.45329]
1.0 [ 0.58032  0.54924  0.49924  0.5169   0.49965]
1.0 [ 0.51987  0.45611  0.61862  0.42451  0.40183]
0.0 [ 0.52375  0.47015  0.61515  0.4301   0.39029]
0.0 [ 0.49196  0.67235  0.5636   0.5913   0.45142]
0.0 [ 0.52992  0.6186   0.53897

0.0 [ 0.31082  0.44404  0.71121  0.35343  0.4163 ]
0.0 [ 0.61785  0.59176  0.54369  0.43306  0.38016]
0.0 [ 0.48834  0.56786  0.58038  0.47051  0.44059]
0.0 [ 0.47664  0.62357  0.56857  0.50354  0.4556 ]
1.0 [ 0.54265  0.38217  0.68077  0.29833  0.28232]
1.0 [ 0.36671  0.51779  0.65113  0.39344  0.44067]
1.0 [ 0.47842  0.56155  0.62175  0.4447   0.40319]
0.0 [ 0.63871  0.86392  0.28263  0.77399  0.73333]
0.0 [ 0.66437  0.59291  0.44334  0.61146  0.53696]
0.0 [ 0.54936  0.58843  0.57387  0.51262  0.40465]
1.0 [ 0.57281  0.31713  0.63844  0.35961  0.38203]
1.0 [ 0.54254  0.50344  0.58379  0.45506  0.41611]
1.0 [ 0.6508   0.27597  0.62045  0.3903   0.34072]
0.0 [ 0.70041  0.65405  0.35905  0.65229  0.61382]
0.0 [ 0.44157  0.51483  0.57803  0.5031   0.48879]
1.0 [ 0.33352  0.46494  0.74637  0.37927  0.32606]
1.0 [ 0.67599  0.43529  0.45069  0.58271  0.56646]
0.0 [ 0.47736  0.60124  0.59343  0.52133  0.43071]
1.0 [ 0.64924  0.71416  0.43214  0.62429  0.52971]
1.0 [ 0.41693  0.43526  0.67564

0.0 [ 0.49783  0.44424  0.61642  0.39929  0.41647]
0.0 [ 0.54794  0.59045  0.55271  0.50755  0.44222]
0.0 [ 0.49843  0.67617  0.5751   0.47231  0.42166]
0.0 [ 0.59681  0.77231  0.41348  0.69205  0.58592]
0.0 [ 0.51656  0.42774  0.62094  0.41796  0.38669]
1.0 [ 0.46072  0.52181  0.65444  0.41549  0.36043]
1.0 [ 0.50271  0.48183  0.64803  0.39291  0.35617]
1.0 [ 0.47953  0.52937  0.61161  0.49457  0.40312]
0.0 [ 0.51206  0.5746   0.58137  0.49996  0.43393]
1.0 [ 0.61327  0.49912  0.58444  0.38839  0.39734]
1.0 [ 0.45417  0.51883  0.64983  0.42264  0.37391]
1.0 [ 0.52308  0.14276  0.73364  0.24938  0.2908 ]
0.0 [ 0.54156  0.45661  0.6377   0.34919  0.32803]
0.0 [ 0.42488  0.49464  0.7266   0.29993  0.29184]
1.0 [ 0.44302  0.42639  0.77773  0.20417  0.16948]
0.0 [ 0.52044  0.49082  0.64385  0.413    0.3467 ]
1.0 [ 0.5901   0.43758  0.52864  0.48103  0.49579]
0.0 [ 0.48434  0.61144  0.56771  0.53767  0.44274]
1.0 [ 0.5144   0.48704  0.64618  0.38214  0.36981]
1.0 [ 0.49542  0.59704  0.58357

0.0 [ 0.46277  0.5862   0.61676  0.46469  0.40588]
0.0 [ 0.48564  0.617    0.57916  0.54254  0.45108]
0.0 [ 0.44529  0.55748  0.59742  0.48842  0.46208]
1.0 [ 0.4072   0.53475  0.66487  0.42185  0.3721 ]
0.0 [ 0.56014  0.51685  0.57562  0.49305  0.42398]
1.0 [ 0.51097  0.58115  0.55965  0.52287  0.46109]
1.0 [ 0.48706  0.55511  0.58428  0.4684   0.44123]
0.0 [ 0.44305  0.36645  0.71469  0.34839  0.31005]
0.0 [ 0.4805   0.54655  0.63244  0.47767  0.37979]
1.0 [ 0.46443  0.61581  0.58683  0.50564  0.44359]
0.0 [ 0.47119  0.32925  0.71292  0.34848  0.31617]
1.0 [ 0.50409  0.52016  0.61921  0.47436  0.39645]
1.0 [ 0.54933  0.39655  0.62908  0.41193  0.36958]
0.0 [ 0.43015  0.65368  0.55047  0.60449  0.53617]
0.0 [ 0.51454  0.54319  0.49065  0.56908  0.59062]
0.0 [ 0.48525  0.54335  0.58037  0.48713  0.44565]
1.0 [ 0.58794  0.58875  0.4693   0.5798   0.53788]
1.0 [ 0.62787  0.70068  0.41412  0.63695  0.55877]
0.0 [ 0.49066  0.49382  0.61031  0.43731  0.40543]
0.0 [ 0.29476  0.46328  0.75163

0.0 [ 0.43889  0.38169  0.67194  0.3577   0.38859]
1.0 [ 0.5016   0.51284  0.60005  0.46161  0.42194]
0.0 [ 0.59299  0.44995  0.52142  0.39572  0.50558]
0.0 [ 0.45846  0.6259   0.62197  0.52958  0.38902]
1.0 [ 0.52955  0.49332  0.57329  0.46812  0.4411 ]
1.0 [ 0.58012  0.56088  0.55652  0.49762  0.42038]
0.0 [ 0.50149  0.46664  0.598    0.4408   0.4435 ]
0.0 [ 0.65868  0.71396  0.33949  0.66873  0.65761]
1.0 [ 0.54525  0.51407  0.59968  0.39841  0.37752]
0.0 [ 0.47514  0.64473  0.55461  0.5072   0.48495]
1.0 [ 0.52407  0.61977  0.53762  0.54141  0.48254]
0.0 [ 0.5021   0.62032  0.57831  0.50551  0.42593]
1.0 [ 0.3366   0.56097  0.69816  0.36783  0.35515]
0.0 [ 0.60345  0.63495  0.53158  0.54235  0.41349]
1.0 [ 0.46369  0.67381  0.50653  0.6229   0.55279]
1.0 [ 0.56342  0.62759  0.49916  0.54026  0.50717]
1.0 [ 0.45166  0.55574  0.62203  0.44677  0.41719]
1.0 [ 0.49506  0.53556  0.62139  0.48113  0.39039]
0.0 [ 0.26445  0.3417   0.74538  0.34109  0.39754]
